# Generate final output 
A postprocessing step that calculates relevant variables and produces the sidewalk map.

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd

import shapely.geometry as sg
import shapely.ops as so
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline

import networkx as nx
import momepy

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import upc_sw.poly_utils as poly_utils

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pickle

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

## Settings

## Import BGT data

In [ ]:
# Read BGT data
df_bgt = gpd.read_file(cf.bgt_road_file)
df_bgt = df_bgt.set_crs(cf.CRS, allow_override=True)
df_bgt = df_bgt.drop_duplicates()

# Remove obstacles (small interiors)
df_bgt['geometry_no_holes'] = df_bgt.progress_apply(
   lambda row: poly_utils.remove_interiors(row.geometry, cf.min_interior_size), axis=1)
df_bgt = df_bgt.set_geometry('geometry_no_holes')

# Merge sidewalk polygons (optional, but should be in line with notebook 5 'merge_sidewalks' parameter)
df_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry_no_holes'].unary_union))
df_bgt = gpd.GeoDataFrame(df_bgt.geometry.explode()) 

# Ignore sidewalk polygons that are too small
df_bgt['area'] = df_bgt['geometry'].area
df_bgt = df_bgt[df_bgt.area > cf.min_area_size]

# Calculate centerlines
df_bgt['centerlines'] = df_bgt.progress_apply(
   lambda row: Centerline(row.geometry, interpolation_distance=0.5), axis=1)
df_bgt = df_bgt.set_geometry('centerlines')

df_bgt['centerlines'] = df_bgt['centerlines'].progress_apply(so.linemerge)
df_bgt['centerlines'] = df_bgt['centerlines'].progress_apply(poly_utils.remove_short_lines)

## Create relevant variable - full width

In [ ]:
# Add sidewalk ID
df_bgt['sidewalk_id'] = range(0, len(df_bgt))

In [ ]:
# Get each centerline in separate row
df_bgt_exp = df_bgt.explode(ignore_index=True)

In [ ]:
# Remove few missing centerlines
df_bgt_exp = df_bgt_exp.dropna()
df_bgt_exp = df_bgt_exp.reset_index()

### Cut lines that are too long

In [ ]:
# Remove geometry column and add length column
df_bgt_cut = df_bgt_exp.drop('geometry', axis=1)
df_bgt_cut['length'] = df_bgt_cut['centerlines'].length

In [ ]:
# Cut linestrings longer than a max length
df_bgt_cut = poly_utils.shorten_linestrings(df_bgt_cut, cf.max_ls_length)

In [ ]:
# Add shortened linestrings to previous dataframe
df_bgt_exp = df_bgt_exp.drop(['centerlines'], axis=1).merge(df_bgt_cut.drop(['area', 'sidewalk_id'], axis=1), 
                                                            how = 'outer', on = 'index')

In [ ]:
del df_bgt_cut

### Get width and color

In [ ]:
df_bgt_exp[['avg_width', 'min_width']] = df_bgt_exp.progress_apply(
    lambda row: poly_utils.get_avg_width_cl(row.geometry, row.centerlines, 
                                            cf.width_resolution, cf.width_precision), axis=1)

In [ ]:
conditions = [
    (df_bgt_exp['min_width'] < cf.width_1),
    (df_bgt_exp['min_width'] >= cf.width_1) & (df_bgt_exp['min_width'] < cf.width_2),
    (df_bgt_exp['min_width'] >= cf.width_2) & (df_bgt_exp['min_width'] < cf.width_3),
    (df_bgt_exp['min_width'] >= cf.width_3) & (df_bgt_exp['min_width'] < cf.width_4),
    (df_bgt_exp['min_width'] >= cf.width_4) & (df_bgt_exp['min_width'] < cf.width_5),
    (df_bgt_exp['min_width'] >= cf.width_5) & (df_bgt_exp['min_width'] < cf.width_6),
    (df_bgt_exp['min_width'] >= cf.width_6)
]

values_color = ['darkred', 'red', 'orange', 'yellow', 'greenyellow', 'limegreen', 'green']
values_indication = ['<' + str(cf.width_1) + 'm', 
                     str(cf.width_1) + '-' + str(cf.width_2) + 'm', 
                     str(cf.width_2) + '-' + str(cf.width_3) + 'm', 
                     str(cf.width_3) + '-' + str(cf.width_5) + 'm', 
                     str(cf.width_4) + '-' + str(cf.width_5) + 'm', 
                     str(cf.width_5) + '-' + str(cf.width_6) + 'm', 
                     '>' + str(cf.width_6) + 'm']

In [ ]:
df_bgt_exp['full_width_color'] = np.select(conditions, values_color)
df_bgt_exp['full_width'] = np.select(conditions, values_indication)
df_bgt_exp['full_width'].value_counts()

### Remove too narrow paths and short-ends

In [ ]:
# Apply minimal path width on BGT centerlines
print(df_bgt_exp.shape)
df_bgt_exp = df_bgt_exp[df_bgt_exp['min_width'] > cf.min_path_width].reset_index(drop=True)
print(df_bgt_exp.shape)

In [ ]:
# Remove short lines
df_bgt_exp = df_bgt_exp.rename(columns={'geometry':'geometry_sidewalks', 'centerlines':'geometry'}) 
mls_per_id = poly_utils.create_mls_per_sidewalk(df_bgt_exp, crs=cf.CRS) 
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(
    lambda x: poly_utils.remove_short_lines(x, cf.min_se_length_fw))

In [ ]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_bgt_exp = df_bgt_exp.merge(long_segments_df, how='inner')
df_bgt_exp.shape

## Import segments data

In [ ]:
# Read lines with widths (calculated in notebook 5)
df_segments = gpd.read_file(cf.segments_file, crs=cf.CRS)

## Create relevant variable - obstacle-free width

### Remove too narrow paths and short-ends

In [ ]:
# Apply minimal path width
df_segments_wide = df_segments[df_segments['min_width'] > cf.min_path_width].reset_index(drop=True)
print(df_segments.shape)
print(df_segments_wide.shape)

In [ ]:
# Remove short lines  
mls_per_id = poly_utils.create_mls_per_sidewalk(df_segments_wide, crs=cf.CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(poly_utils.remove_short_lines)

In [ ]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_segments_wide = df_segments_wide.merge(long_segments_df, how='inner')
df_segments_wide.shape

### Use color codes

In [ ]:
conditions = [
    (df_segments_wide['min_width'] < cf.width_1),
    (df_segments_wide['min_width'] >= cf.width_1) & (df_segments_wide['min_width'] < cf.width_2),
    (df_segments_wide['min_width'] >= cf.width_2) & (df_segments_wide['min_width'] < cf.width_3),
    (df_segments_wide['min_width'] >= cf.width_3) & (df_segments_wide['min_width'] < cf.width_4),
    (df_segments_wide['min_width'] >= cf.width_4) & (df_segments_wide['min_width'] < cf.width_5),
    (df_segments_wide['min_width'] >= cf.width_5) & (df_segments_wide['min_width'] < cf.width_6),
    (df_segments_wide['min_width'] >= cf.width_6)
]

values = ['darkred', 'red', 'orange', 'yellow', 'greenyellow', 'limegreen', 'green']

In [ ]:
df_segments_wide['min_width_color'] = np.select(conditions, values)
df_segments_wide['min_width_color'].value_counts()

In [ ]:
# Add width factor, for calculating the weights of the paths later
#values = [1000000000, 1000000, 1000, 1]
values = [1000000000000, 10000000000, 100000000, 1000000, 10000, 100, 1]
df_segments_wide['min_width_factor'] = np.select(conditions, values).astype('int64')

### Take point cloud coverage into account

In [ ]:
df_segments_wide['min_width_factor'][df_segments_wide['pc_coverage'] == False] = 100000000000001

In [ ]:
df_segments_wide['min_width_factor'].value_counts()

### Do network calculation

In [ ]:
df_bgt_exp['route_weight'] = np.nan

In [ ]:
# Create final dataframe
final_df = pd.DataFrame()

for i in range(len(df_bgt['sidewalk_id'])):  
      
    # Get sidewalk polygon for this centerline 
    my_sidewalk = df_bgt['geometry'].values[i]   
    
    # Create graph for all paths withing this sidewalk polygon
    df_sidewalk = df_segments_wide[df_segments_wide['geometry'].within(my_sidewalk)].reset_index(drop=True)
    G = momepy.gdf_to_nx(df_sidewalk, approach="primal", multigraph=True)
    
    # Create dataframe with linestrings of centerline
    centerline_df = df_bgt_exp[df_bgt_exp['sidewalk_id'] == i].reset_index(drop=True)
    
    for j in range(len(centerline_df['geometry'])):  
        
        # Get line
        my_line = centerline_df.iloc[[j]]['geometry'].values[0]
        
        if len(list(G.nodes)) > 0:
            # Check if my_line has start and end (not a ring)
            if len(my_line.boundary) == 0:
                print('no route calculated for line (j)', j, 'in sidewalk (i)', i, '(ring)')
                centerline_df['route_weight'][j] = 0
            else:
                # Get origin and destination location
                origin_point, dest_point = my_line.boundary
                origin_node_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G.nodes)))[1]
                dest_node_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G.nodes)))[1]

                # Get origin and destination node
                origin_node = (origin_node_loc.x, origin_node_loc.y)
                dest_node = (dest_node_loc.x, dest_node_loc.y)   

                # Get weight of optimal route in graph
                try:
                    route_weight = nx.shortest_path_length(G, origin_node, dest_node,
                                                           weight='min_width_factor')
                    if (origin_point.distance(origin_node_loc) < cf.max_dist) and (dest_point.distance(dest_node_loc) < cf.max_dist):
                        centerline_df['route_weight'][j] = route_weight
                    else:
                        print('origin and/or destination node too far from line start/end for line (j)', j, 'in sidewalk (i)', i) 
                        centerline_df['route_weight'][j] = np.nan
                except nx.NetworkXNoPath:
                    print('no route found for line (j)', j, 'in sidewalk (i)', i)
                    centerline_df['route_weight'][j] = 100000000000000
        else:
            print('network has zero nodes')

    # Append data to final dataframe
    final_df = final_df.append(centerline_df)
final_df = final_df.reset_index()

In [ ]:
del df_bgt_exp

In [ ]:
# Get final color of the routes
final_df['final_color'] = final_df.progress_apply(
    lambda row: poly_utils.get_route_color(row.route_weight), axis=1)

In [ ]:
final_df_group = final_df.groupby('final_color').sum()
final_df_group['length_perc'] = round(100 * final_df_group['length'] / sum(final_df_group['length']), 1)
final_df_group[['length_perc']]

## Post-process 

In [ ]:
final_df = final_df.set_geometry('geometry')

### Set final color and indication

In [ ]:
# Get dataframe with only valid lines
final_df_select = final_df[final_df['final_color'].isin(['green', 'limegreen', 'greenyellow', 
                                                         'yellow', 'orange', 'red',
                                                         'darkred', 'purple', 'black', 'grey'])]

final_df_select = final_df_select.reset_index(drop=True)

In [ ]:
# Recolor paths with unknown widths
final_df_select['final_color'] = final_df_select['final_color'].replace(['black', 'purple'], 'grey')

In [ ]:
# Add meter indication
conditions = [
    (final_df_select['final_color'] == 'darkred'),
    (final_df_select['final_color'] == 'red'),   
    (final_df_select['final_color'] == 'orange'),
    (final_df_select['final_color'] == 'yellow'),
    (final_df_select['final_color'] == 'greenyellow'),
    (final_df_select['final_color'] == 'limegreen'),  
    (final_df_select['final_color'] == 'green'),  
    (final_df_select['final_color'] == 'grey')
]
values_indication = ['<' + str(cf.width_1) + 'm', 
                     str(cf.width_1) + '-' + str(cf.width_2) + 'm', 
                     str(cf.width_2) + '-' + str(cf.width_3) + 'm', 
                     str(cf.width_3) + '-' + str(cf.width_4) + 'm', 
                     str(cf.width_4) + '-' + str(cf.width_5) + 'm', 
                     str(cf.width_5) + '-' + str(cf.width_6) + 'm', 
                     '>' + str(cf.width_6) + 'm',
                    'unknown']
final_df_select['obstacle_free_width'] = np.select(conditions, values_indication)

In [ ]:
final_df_select.shape

### Get correct columns

In [ ]:
# Select only relevant columns
final_df_select = final_df_select[['geometry', 'final_color', 'obstacle_free_width', 
                                   'full_width', 'full_width_color']]

In [ ]:
# Add object ID
final_df_select['object_id'] = final_df_select.index

### Fix incorrect labeling of obstacle-free width (> full width)

In [ ]:
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['0.9-1.5m', '1.5-2.0m', '2.0-2.2m', '2.2-2.9m', '2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '<0.9m')] = '<0.9m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['1.5-2.0m', '2.0-2.2m', '2.2-2.9m', '2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '0.9-1.5m')] = '0.9-1.5m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['2.0-2.2m', '2.2-2.9m', '2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '1.5-2.0m')] = '1.5-2.0m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['2.2-2.9m', '2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '2.0-2.2m')] = '2.0-2.2m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'].isin(['2.9-3.6m', '>3.6m'])) & 
                                       (final_df_select['full_width'] == '2.2-2.9m')] = '2.2-2.9m'
final_df_select['obstacle_free_width'][(final_df_select['obstacle_free_width'] == '>3.6m') & 
                                       (final_df_select['full_width'] == '2.9-3.6m')] = '2.9-3.6m'

### Adjust crs

In [ ]:
final_df_select = final_df_select.set_crs(cf.CRS) 

## Store final output

In [ ]:
final_df_select.to_file(cf.output_file, driver='GPKG')

## Visualizations

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5), dpi=2000, 
                                    frameon=False, constrained_layout=True)

# Plot process step 1
df_bgt.set_geometry('geometry').plot(ax=ax1, color="silver")
df_segments_wide.plot(ax=ax1, linewidth=0.1, color=df_segments_wide['min_width_color'])
ax1.axis('off')

# Plot process step 2
df_bgt.set_geometry('geometry').plot(ax=ax2, color="silver")
final_df.plot(ax=ax2, linewidth=0.5, color=final_df.final_color)  
ax2.axis('off')

# Plot process step 3
df_bgt.set_geometry('geometry').plot(ax=ax3, color="silver")
final_df_select.plot(ax=ax3, linewidth=0.5, color=final_df_select.final_color)  
ax3.axis('off')

# Create legends
wa = mpatches.Patch(color='green', label='>' + str(cf.width_6) + 'm')
wb = mpatches.Patch(color='limegreen', label= str(cf.width_5) + '-' + str(cf.width_6) + 'm')
wc = mpatches.Patch(color='greenyellow', label= str(cf.width_4) + '-' + str(cf.width_5) + 'm')
wd = mpatches.Patch(color='yellow', label= str(cf.width_3) + '-' + str(cf.width_4) + 'm')
we = mpatches.Patch(color='orange', label= str(cf.width_2) + '-' + str(cf.width_3) + 'm')
wf = mpatches.Patch(color='red', label= str(cf.width_1) + '-' + str(cf.width_2) + 'm')
wg = mpatches.Patch(color='darkred', label='<' + str(cf.width_1) + 'm')
rno = mpatches.Patch(color='brown', label='no route possible')
rnan = mpatches.Patch(color='purple', label='node too far')
rl0 = mpatches.Patch(color='black', label='route length 0')
rnpc = mpatches.Patch(color='grey', label='no pc coverage')
runk = mpatches.Patch(color='grey', label='unknown')

legend1 = plt.legend(handles=[wa,wb,wc,wd,we,wf,wg,rno,rnan,rl0,rnpc], 
           bbox_to_anchor=(-0.5, -0.5, 0.5, 0.5))
plt.legend(handles=[wa,wb,wc,wd,we,wf,wg,runk], 
           bbox_to_anchor=(0.5, -0.5, 0.5, 0.5))
plt.gca().add_artist(legend1)

plt.savefig(cf.output_image, bbox_inches='tight')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5), dpi=2000, 
                                    frameon=False, constrained_layout=True)

# Plot process step 3 - OFW
df_bgt.set_geometry('geometry').plot(ax=ax1, color="silver")
final_df_select.plot(ax=ax1, linewidth=0.5, color=final_df_select.final_color)  
ax1.axis('off')

# Plot process step 3 - FW
df_bgt.set_geometry('geometry').plot(ax=ax2, color="silver")
final_df_select.plot(ax=ax2, linewidth=0.5, color=final_df_select.full_width_color)  
ax2.axis('off')

# Create legends
wa = mpatches.Patch(color='green', label='>' + str(cf.width_6) + 'm')
wb = mpatches.Patch(color='limegreen', label= str(cf.width_5) + '-' + str(cf.width_6) + 'm')
wc = mpatches.Patch(color='greenyellow', label= str(cf.width_4) + '-' + str(cf.width_5) + 'm')
wd = mpatches.Patch(color='yellow', label= str(cf.width_3) + '-' + str(cf.width_4) + 'm')
we = mpatches.Patch(color='orange', label= str(cf.width_2) + '-' + str(cf.width_3) + 'm')
wf = mpatches.Patch(color='red', label= str(cf.width_1) + '-' + str(cf.width_2) + 'm')
wg = mpatches.Patch(color='darkred', label='<' + str(cf.width_1) + 'm')
runk = mpatches.Patch(color='grey', label='unknown')

legend1 = plt.legend(handles=[wa,wb,wc,wd,we,wf,wg,runk], 
           bbox_to_anchor=(-0.5, -0.5, 0.5, 0.5), title = "obstacle free width")
plt.legend(handles=[wa,wb,wc,wd,we,wf,wg], 
           bbox_to_anchor=(0.5, -0.5, 0.5, 0.5), title = "full width")
plt.gca().add_artist(legend1)

plt.savefig(cf.output_image_no, bbox_inches='tight')
plt.show()